# CIVL 440 - Assignment 4
Name: Miguel Pampolina\
Date: Nov 13, 2022

In [1]:
import numpy as np
import pandas as pd
import math
from icecream import ic

### 1. Intersection Parameters

Notes:
- No adjustment factor has been made with respect to the lane width, as all lanes have a width of 4m which is between the threshold levels of 3m to 4.4m. (see pg. 3-33 / pg. 53)
- No adjustment factor has been made with respect to the grade of the intersection as the 1% grade of the intersection of interest is between the threshold levels of +2% to -2%. (see pg. 3-34 / pg. 54)
- No adjustment factor has been made with respect to the curb radii, as all approaches have a radii of of 16m which is greater than 15m threshold. (see pg. 3-35 / pg. 55)
- No adjustment factor has been made with respect to buses, as all bus stops are located at downstream intersections. (see pg. 3-38 / pg. 58)

In [2]:
# Intersection parameters
# Demands in pcu/hr
DEMANDS = {
    "nb":{
        "LT": 350,
        "TH": 208,
        "RT": 50,
        "PED": 120,
    },
    "eb":{
        "LT": 150,
        "TH": 162,
        "RT": 215,
        "PED": 100
    },
    "sb":{
        "LT": 350,
        "TH": 216,
        "RT": 200,
        "PED": 30
    },
    "wb":{
        "LT": 180,
        "TH": 210,
        "RT": 250,
        "PED": 50
    }
}
LANE_CONFIG = {
    "nb": ["LT", "TH-RT"],
    "eb": ["LT", "TH-RT"],
    "sb": ["LT", "TH-RT"],
    "wb": ["LT", "TH-RT"]
}
# Receiving lanes by approach
RE_LN = {
    "s": 1,
    "w": 1,
    "n": 1,
    "e": 1
}
CUR_RAD = 16 #metres
GRADE = 0 # FIXME
ON_STREET_PARKING = False
RTOR = False
S0 = 1650 #pcu/hr
C_INITIAL = 120 #s
PED_SPEED = 1.2 #m/s

### 2. Intersection Independent Parameters

In [3]:
# Intersection Independent Parameters
# Left-turn opposing movements
LT_OPPOSE_MVMT = ["TH", "RT"]
# Opposing directions lookup
OPPOSE_DIR = {
    "nb": "sb",
    "sb": "nb",
    "eb": "wb",
    "wb": "eb"
}
# Left-turn movement direction to receiving approach lookup
LT_RE_LN = {
    "nb": "w",
    "sb": "e", 
    "eb": "n",
    "wb": "s"
}

### 3. Phasing Functions

In [4]:
def getLanes(laneConfig, dir, mvmt, opposing):
    count = 0
    analysis_dir = dir
    if opposing:
        analysis_dir = OPPOSE_DIR[dir]
        for lane in laneConfig[analysis_dir]:
            if "TH" in lane:
                count += 1
            elif "RT" in lane:
                count += 1
    else:
        for lane in laneConfig[analysis_dir]:
            if mvmt in lane:
                count += 1
    return count

In [5]:
# Table 3-4: Number of cross-street lanes available for waiting vehicles; Quantity in pcu that can discharge during one intergreen period.
X_LTOI = {
    1: 0.75,
    2: 1.5,
    3: 2.5
}
'''
Returns the quantity of pcus per hour that complete the left turn in the intergreen period
    Parameters:
            rec_lanes (float): Number of receiving lanes
            c (float): Cycle length
    Returns:
            q_ltoi (float): Quantity of left turns of intergreen (pcu/hr)
'''
def get_flowLTOI(rec_lanes, c):
    return (3600/c) * X_LTOI[rec_lanes]

In [6]:
# Table 3-16: Effect of Number of Lanes on Permissive Left-Turn Saturation Flow Rate
F_OPP = {
    1: 1,
    2: 0.625,
    3: 0.51,
    4: 0.44
}
'''
Returns the saturation flow of the left-turn permissive phase in pcu/hr.
    Parameters:
            dir (str): Movement direction (e.g. "nb")
            demands (dict): Demands for all movements
            laneConfig (dict): Lane configuration
            c (float): cycle length
            g: green time for the current phase
            s0: base saturation flow rate
    Returns:
            satFlowPermLT (float): saturation flow of the left-turn permissive phase (pcu/hr)
'''
def get_satFlowPermLT(dir, demands, laneConfig, c, g, s0):
    opposingDirection = OPPOSE_DIR[dir]
    # FIXME This would technically be dependent on the number of receiving lanes
    opposingDemand = demands[opposingDirection].get("RT", 0) + demands[opposingDirection].get("TH", 0)
    opposingDemandPeds = demands[opposingDirection].get("PED", 0)
    # FIXME Pedestrian F-factor
    q0 = (F_OPP[getLanes(laneConfig, dir, "LT", True)]*opposingDemand*(c/g))+(opposingDemandPeds*(c/g))
    return s0*(1.05*np.exp(-0.00121*q0)-0.05)

In [7]:
def get_satFlowTHLT(dir, demands, s0, satFlowPermLT):
    Klt = s0/satFlowPermLT
    Fthlt = (demands[dir].get("LT", 0) + demands[dir].get("TH", 0))/(Klt * demands[dir].get("LT", 0) + demands[dir].get("TH", 0))
    return s0 * Fthlt

In [8]:
# Note: This does not account for lane configurations with a through lane + a through right lane
def get_satFlowTHRT(dir, demands, c, g, s0):
    qPrimePed = demands[dir].get("PED", 0)*(c/g)
    if qPrimePed <= 200:
        FRPed = 1
    else:
        FRPed = 0.44-qPrimePed/14100 # Vancouver
    satFlowRT = s0 * FRPed
    Kr = s0/satFlowRT
    Fthrt = (demands[dir].get("TH", 0) + demands[dir].get("RT", 0))/(Kr * demands[dir].get("RT", 0) + demands[dir].get("TH", 0))
    return s0 * Fthrt

In [9]:
def get_yCritLT(dir, mvmt, arrow, demands, laneConfig, rec_lanes, c, g_perm, s0, prot):
    satFlowPermLT = get_satFlowPermLT(dir, demands, laneConfig, c, g_perm, s0)
    satFlowTHLT = get_satFlowTHLT(dir, demands, s0, satFlowPermLT)
    flowLTOI = get_flowLTOI(rec_lanes, c)
    satFlow = s0
    if arrow == "LT" and prot:
        return (demands[dir][mvmt] - flowLTOI - satFlowPermLT*(g_perm/c)) / satFlow
    elif arrow == "LT" and not prot:
        return (demands[dir][mvmt] - flowLTOI) / satFlowPermLT
    elif arrow == "LT-TH" or arrow == "LT-TH-TR":
        return (demands[dir].get("LT", 0) + demands[dir].get("TH", 0)) / satFlowTHLT

In [10]:
def get_yCritTHRT(dir, demands, c, g, s0):
    return (demands[dir].get("TH", 0) + demands[dir].get("RT", 0)) / get_satFlowTHRT(dir, demands, c, g, s0)

In [11]:
# !Custom Function (alter if changing the phasing structure)
def four_phase_plan(c, g1, g2, g3, g4, num_phase=4):
    for _ in range(1000):
        yCritLT_EB = get_yCritLT("eb", "LT", "LT", DEMANDS, LANE_CONFIG, RE_LN[LT_RE_LN["eb"]], c, g1, S0, True)
        yCritLT_WB = get_yCritLT("wb", "LT", "LT", DEMANDS, LANE_CONFIG, RE_LN[LT_RE_LN["wb"]], c, g1, S0, True)
        y1Max = max(yCritLT_EB, yCritLT_WB)
        yCritTH_EB = get_yCritTHRT("eb", DEMANDS, c, g2, S0)
        yCritTH_WB = get_yCritTHRT("wb", DEMANDS, c, g2, S0)
        y2Max = max(yCritTH_EB, yCritTH_WB)
        yCritLT_NB = get_yCritLT("nb", "LT", "LT", DEMANDS, LANE_CONFIG, RE_LN[LT_RE_LN["nb"]], c, g3, S0, True)
        yCritLT_SB = get_yCritLT("sb", "LT", "LT", DEMANDS, LANE_CONFIG, RE_LN[LT_RE_LN["sb"]], c, g3, S0, True)
        y3Max = max(yCritLT_NB, yCritLT_SB)
        yCritTH_NB = get_yCritTHRT("nb", DEMANDS, c, g4, S0)
        yCritTH_SB = get_yCritTHRT("sb", DEMANDS, c, g4, S0)
        y4Max = max(yCritTH_NB, yCritTH_SB)
        Y = y1Max + y2Max + y3Max + y4Max
        lostTime = num_phase * (3 + 1 - 1)
        Copt = (1.5*lostTime+5)/(1-Y)
        G = Copt - (num_phase*(3+1))
        Cmin = lostTime/(1-Y)
        g1_sug = (y1Max/Y)*G
        g2_sug = (y2Max/Y)*G
        g3_sug = (y3Max/Y)*G
        g4_sug = (y4Max/Y)*G
        g1 = g1_sug
        g2 = g2_sug
        g3 = g3_sug
        g4 = g4_sug
        c = Copt
    ic("Protected Left Turns EW:")
    ic(yCritLT_EB)
    ic(yCritLT_WB)
    ic("Protected Through EW:")
    ic(yCritTH_EB)
    ic(yCritTH_WB)
    ic("Protected Left Turns NS:")
    ic(yCritLT_NB)
    ic(yCritLT_SB)
    ic("Protect Through NS:")
    ic(yCritTH_NB)
    ic(yCritTH_SB)
    ic("Sum Y:")
    ic(Y)
    ic(g1)
    ic(g2)
    ic(g3)
    ic(g4)
    ic(g1_sug)
    ic(g2_sug)
    ic(g3_sug)
    ic(g4_sug)
    ic(Cmin)

In [12]:
# !Custom Function (alter if changing the phasing structure)
def three_phase_plan(c, g1, g2, g3, num_phase=3):
    for _ in range(1000):
        yCritLT_EB = get_yCritLT("eb", "LT", "LT", DEMANDS, LANE_CONFIG, RE_LN[LT_RE_LN["eb"]], c, g1, S0, False)
        yCritLT_WB = get_yCritLT("wb", "LT", "LT", DEMANDS, LANE_CONFIG, RE_LN[LT_RE_LN["wb"]], c, g1, S0, False)
        yCritTH_EB = get_yCritTHRT("eb", DEMANDS, c, g1, S0)
        yCritTH_WB = get_yCritTHRT("wb", DEMANDS, c, g1, S0)
        y1Max = max(yCritLT_EB, yCritLT_WB, yCritTH_EB, yCritTH_WB)
        yCritLT_NB = get_yCritLT("nb", "LT", "LT", DEMANDS, LANE_CONFIG, RE_LN[LT_RE_LN["nb"]], c, g2, S0, True)
        yCritLT_SB = get_yCritLT("sb", "LT", "LT", DEMANDS, LANE_CONFIG, RE_LN[LT_RE_LN["sb"]], c, g2, S0, True)
        y2Max = max(yCritLT_NB, yCritLT_SB)
        yCritTH_NB = get_yCritTHRT("nb", DEMANDS, c, g3, S0)
        yCritTH_SB = get_yCritTHRT("sb", DEMANDS, c, g3, S0)
        y3Max = max(yCritTH_NB, yCritTH_SB)
        Y = y1Max + y2Max + y3Max
        lostTime = num_phase * (3 + 1 - 1)
        Copt = (1.5*lostTime+5)/(1-Y)
        G = Copt - (3*(3+1))
        Cmin = lostTime/(1-Y)
        g1_sug = (y1Max/Y)*G
        g2_sug = (y2Max/Y)*G
        g3_sug = (y3Max/Y)*G
        g1 = g1_sug
        g2 = g2_sug
        g3 = g3_sug
        c = Copt
    ic("Protected Left Turns & Through EW:")
    ic(yCritLT_EB)
    ic(yCritLT_WB)
    ic(yCritTH_EB)
    ic(yCritTH_WB)
    ic("Protected Left Turns NS:")
    ic(yCritLT_NB)
    ic(yCritLT_SB)
    ic("Protected Through NS:")
    ic(yCritTH_NB)
    ic(yCritTH_SB)
    ic("Sum Y:")
    ic(Y)
    ic(g1)
    ic(g2)
    ic(g3)
    ic(c)
    ic(g1_sug)
    ic(g2_sug)
    ic(g3_sug)
    ic(Cmin)

### 4. Delay Functions

In [13]:
def delay_to_los(delay):
    if delay < 10:
        return "A"
    elif delay > 10 and delay <= 20:
        return "B"
    elif delay > 20 and delay <= 35:
        return "C"
    elif delay > 35 and delay <= 55:
        return "D"
    elif delay > 55 and delay <= 80:
        return "E"
    else:
        return "F"

In [14]:
def get_delay(satFlow, flowLTOI, demand, c, g, te, FLane=1):
    capacity = (satFlow * (g/c)) + flowLTOI
    x1 = min(FLane, (demand/capacity))
    d1 = (c*(1-g/c)**2)/(2*(1-x1*(g/c)))
    d2 = 15*te*((x1-1)+math.sqrt((1-x1)**2+((240*x1)/(capacity*te))))
    return d1 + d2

### 5. Pedestrian Clearance Time
- For all through movements, the walk interval will be 10s in accordance with the minimums described by the CCGSI (see pg. 3-69 / pg. 89)
- FDW intervals for the through movements are calculated as follows (see pg. 3-70 / pg. 90)

In [15]:
w = 10 #s
ebFDW = 12/PED_SPEED #s
wbFDW = 15/PED_SPEED #s
nbFDW = 15/PED_SPEED #s
sbFDW = 12/PED_SPEED #s
print("Four Phase Structure:")
print(f"Min Pedestrian Clearance Phase 2 [eb-wb]: {w + max(ebFDW, wbFDW)}s")
print(f"Min Pedestrian Clearance Phase 4 [nb-sb]: {w + max(nbFDW, sbFDW)}s")
print("\nThree Phase Structure:")
print(f"Min Pedestrian Clearance Phase 1 [eb-wb]: {w + max(ebFDW, wbFDW)}s")
print(f"Min Pedestrian Clearance Phase 3 [nb-sb]: {w + max(nbFDW, sbFDW)}s")

Four Phase Structure:
Min Pedestrian Clearance Phase 2 [eb-wb]: 22.5s
Min Pedestrian Clearance Phase 4 [nb-sb]: 22.5s

Three Phase Structure:
Min Pedestrian Clearance Phase 1 [eb-wb]: 22.5s
Min Pedestrian Clearance Phase 3 [nb-sb]: 22.5s


### 6. 4-Phase Signal Timing Plan - Divergent Solution

In [16]:
four_phase_plan(120, 19, 61, 20, 20)

ic| 'Protected Left Turns EW:'
ic| yCritLT_EB: 0.09433124544984572
ic| yCritLT_WB: 0.1122979014323676
ic| 'Protected Through EW:'
ic| yCritTH_EB: 0.4064583171469009
ic| yCritTH_WB: 0.2787878787878788
ic| 'Protected Left Turns NS:'
ic| yCritLT_NB: 0.20485116670504597
ic| yCritLT_SB: 0.19708136538725096
ic| 'Protect Through NS:'
ic| yCritTH_NB: 0.20060726455253333
ic| yCritTH_SB: 0.25212121212121213
ic| 'Sum Y:'
ic| Y: 0.9757285974055265
ic| g1: 107.22106885895785
ic| g2: 388.08289963771915
ic| g3: 195.59012920956334
ic| g4: 240.72316134895087
ic| g1_sug: 107.22106885895785
ic| g2_sug: 388.08289963771915
ic| g3_sug: 195.59012920956334
ic| g4_sug: 240.72316134895087
ic| Cmin: 494.40900472444747


### 7. 3-phase Signal Timing Plan - Convergent Solution

In [17]:
three_phase_plan(120, 36, 36, 36)

ic| 'Protected Left Turns & Through EW:'
ic| yCritLT_EB: 0.3676896488051088
ic| yCritLT_WB: 0.40527020080099607
ic| yCritTH_EB: 0.4053084882636943
ic| yCritTH_WB: 0.2787878787878788
ic| 'Protected Left Turns NS:'
ic| yCritLT_NB: 0.19623257833600793
ic| yCritLT_SB: 0.18876902191827502
ic| 'Protected Through NS:'
ic| yCritTH_NB: 0.2003573421147779
ic| yCritTH_SB: 0.25212121212121213
ic| 'Sum Y:'
ic| Y: 0.8536622787209145
ic| g1: 54.325179408869886
ic| g2: 26.30186717686774
ic| g3: 33.79285279699252
ic| c: 126.41989938273016
ic| g1_sug: 54.325179408869886
ic| g2_sug: 26.30186717686774
ic| g3_sug: 33.79285279699252
ic| Cmin: 61.501572672679536


In [18]:
54+26+34+12

126

In [19]:
# !Custom Function (alter if changing the phasing structure)
def get_averageDelayThreePhase(phaseList, c, s0, demands, laneConfig, te, gPerm):
    for i, phase in enumerate(phaseList, 1):
        df = {}
        for mvmt, dir, g, row, in phase:
            if mvmt == "LT":
                satFlow = get_satFlowPermLT(dir, demands, laneConfig, c, g, s0)
                flowLTOI = get_flowLTOI(RE_LN[LT_RE_LN[dir]], c)  
                if row == "PermOnly":
                    demand = demands[dir].get("LT", 0)
                elif row == "Prot":
                    demand = demands[dir].get("LT", 0) - flowLTOI - satFlow * (gPerm/c)
                    satFlow = s0
                elif row == "Perm":
                    demand = satFlow * g/c
            elif mvmt == "TH-RT":
                satFlow = get_satFlowTHRT(dir, demands, c, g, s0)
                flowLTOI = 0
                demand = demands[dir].get("TH", 0) + demands[dir].get("RT", 0)
            df[f"{dir}-{mvmt}"] = [round(get_delay(satFlow, flowLTOI, demand, c, g, te), 1)]
        print(f"Phase ({i}) Delay in seconds:")
        print(pd.DataFrame(df, index=["Delay (s)"]))


In [20]:
phaseList = [
    [("LT", "eb", 54, "PermOnly"), ("TH-RT", "eb", 54, "Prot"), ("LT", "wb", 54,  "PermOnly"), ("TH-RT", "wb", 54, "Prot")],
    [("LT", "nb", 26, "Prot"), ("LT", "sb", 26, "Prot")],
    [("LT", "nb", 34, "Perm"), ("TH-RT", "nb", 34, "Prot"), ("LT", "sb", 34, "Perm"), ("TH-RT", "sb", 34, "Prot")]
]

In [21]:
get_averageDelayThreePhase(phaseList, 126, S0, DEMANDS, LANE_CONFIG, 60, 34)

Phase (1) Delay in seconds:
           eb-LT  eb-TH-RT  wb-LT  wb-TH-RT
Delay (s)  118.9      86.9  170.5      33.2
Phase (2) Delay in seconds:
           nb-LT  sb-LT
Delay (s)   79.0   70.3
Phase (3) Delay in seconds:
           nb-LT  nb-TH-RT  sb-LT  sb-TH-RT
Delay (s)   91.8      56.4   98.0      87.3
